In [170]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
from keras.callbacks import ModelCheckpoint
from keras import optimizers
import numpy as np
import random
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional

import sys
import os
from keras.layers import Dense, Activation, Dropout

In [171]:
CORPUS_PATH = os.path.join('data/sample/002')
#CORPUS_PATH = os.path.join('data/001')
filenames1 = sorted([os.path.join(CORPUS_PATH, fn) for fn in os.listdir(CORPUS_PATH)])[1:]

In [172]:
text = ' '.join([open(i).read().lower() for i in filenames1])
chars = sorted(set(list(' '.join(text))))

In [173]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [174]:
print(len(char_indices))

51


In [175]:
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 1633


In [176]:
print('Vectorization...')
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

Vectorization...


In [177]:
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [178]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(200))
model.add(Dropout(0.2))
model.add(Dense(len(chars)))

model.add(Activation('softmax'))
adam = optimizers.Adam(lr=0.01)

model.compile(loss='categorical_crossentropy', optimizer=adam)

In [179]:
model.fit(X, y, batch_size=128, nb_epoch=10, validation_split=0.2)
checkpointer = ModelCheckpoint(filepath="models" + str(i) + ".hdf5", verbose=1, save_best_only=True)

/home/ubuntu/anaconda2/envs/fastai/lib/python3.6/site-packages/keras/models.py:874: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 1306 samples, validate on 327 samples
Epoch 1/10
1306/1306 [==============================] - 3s 2ms/step - loss: 3.4437 - val_loss: 3.1158
Epoch 2/10
1306/1306 [==============================] - 1s 1ms/step - loss: 2.9534 - val_loss: 3.0426
Epoch 3/10
1306/1306 [==============================] - 1s 1ms/step - loss: 2.7999 - val_loss: 2.8752
Epoch 4/10
1306/1306 [==============================] - 1s 1ms/step - loss: 2.6063 - val_loss: 2.7632
Epoch 5/10
1306/1306 [==============================] - 1s 1ms/step - loss: 2.3996 - val_loss: 2.7065
Epoch 6/10
1306/1306 [==============================] - 1s 1ms/step - loss: 2.2582 - val_loss: 2.6796
Epoch 7/10
1306/1306 [==============================] - 1s 1ms/step - loss: 2.1152 - val_loss: 2.6629
Epoch 8/10
1306/1306 [==============================] - 1s 1ms/step - loss: 1.9996 - val_loss: 2.6401
Epoch 9/10
1306/1306 [==============================] - 1s 1ms/step - loss: 1.8772 - val_loss: 2.6700
Epoch 10/10
1306/1306 [============

In [180]:
model.save('final_model.hdf5')

In [181]:
start_index = random.randint(0, len(text) - maxlen - 1)

for diversity in [0.2, 0.5, 1.0, 1.2]:
    print()
    print('----- diversity:', diversity)

    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(400):
        x = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x[0, t, char_indices[char]] = 1.

        preds = model.predict(x, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)


----- diversity: 0.2
----- Generating with seed: "a species, and,
instituting a “test and "
a species, and,
instituting a “test and the and the and rometeres of cod ar or of the and the and the and the and the and the and the and the and the all and or or the and the and the and the and the and the and the and the alin the and the and the and ar of the are of the and the and the and the all tere of the and the and the and the all tes and the all and or or acs on the an the and the and the all ares of the and ard or the all ses
----- diversity: 0.5
----- Generating with seed: "a species, and,
instituting a “test and "
a species, and,
instituting a “test and the on the ave the alua the aly ar ar or acor rsurd act reromutit an the allis ang an the nof the ar abell anf the armeation ar the on the and the the arme the an fourite the and teve alut ing for tha the ano durestent alis and cor cor ar od and te al dereace the alu tes and or coments tian in acteres productss od act and aly the th

In [144]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)